# 对抗FGSM

本节使用快速梯度符号方法Fast Gradient Signed Method (FGSM)攻击 生成 *对抗样本adversarial example* ，参考文献[Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572). 这是第一个也是最流行的欺骗神经网络的攻击。 

## 什么是对抗样本?

对抗样本是故意生成欺骗神经网络的输入，它使得网络错误分类。这些对抗输入对于人眼是不可分别的，但是会使得网络错误识别图像内容。有多种这种类型的攻击，但是，这里关注快速梯度符号方法攻击，它是一种*白盒*攻击方法，目标是产生错误分类。白盒攻击是对攻击模型可以完全访问。下面是一个来自论文的一个最有名的对抗样本。

![Adversarial Example](adversarial_example.png)

这里，从熊猫图像开始，攻击像原始图像加入小的扰动，结果导致图像被分类为长臂猿，而且置信度还比较高。加入扰动的过程下面解释。

## 快速梯度符号法Fast gradient sign method

FGSM使用神经网络的梯度产生一个对抗样本。对于输入图像，方法使用输入图像损失的梯度产生一个新图像，新图像使得损失最大。这个新图像称为对抗图像。如下公式所示：

$$adv\_x = x + \epsilon*\text{sign}(\nabla_xJ(\theta, x, y))$$

其中 

*   adv_x : 对抗图像.
*   x : 原始输入图像.
*   y : 原始图像标签.
*   $\epsilon$ : 确保扰动足够小的系数.
*   $\theta$ : 模型参数.
*   $J$ : 损失.

一个有趣的性质是梯度是对输入图像的，这是因为目标是创建一个最大化损失的图像。完成这个目标的一个方法是，确定图像中每个点对损失值的贡献，然后相应增加扰动。这个方法运行相当快，因为通过使用链式法则和寻找需要的梯度可以容易的确定每个输入像素对损失的贡献。因此，梯度是相对于图像的。而且，模型不再训练（因而梯度不是相对于可训练变量，即模型参数），模型参数保持不变。唯一的目标是欺骗已经训练好的模型。

因此，我们试着愚弄一个预训练的模型，在本节，模型是 [MobileNetV2](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications/MobileNetV2) , 它是在[ImageNet](http://www.image-net.org/)上训练好的.

In [ ]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

让我们加载预训练的 MobileNetV2 模型和ImageNet类标签.

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet标签
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
# 图像预处理函数，使得图像处理后可以喂给MobileNetV2
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# 从概念向量中提取标签的函数
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

## 原始图像

让我们使用一张[Labrador Retriever](https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg) 图像， 并从它生成对抗样本. 第一步是对它预处理，使得它可以作为MobileNetV2模型的输入。

In [ ]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image = tf.image.decode_image(image_raw)

image = preprocess(image)
image_probs = pretrained_model.predict(image)

让我们看看这种图片。

In [ ]:
plt.figure()
plt.imshow(image[0]*0.5+0.5) # To change [-1, 1] to [0,1]
_, image_class, class_confidence = get_imagenet_label(image_probs)
plt.title('{} : {:.2f}% Confidence'.format(image_class, class_confidence*100))
plt.show()

## 创建对抗样本

### 使用FGSM
第一步，创建用来对原始图像修改的扰动。如前所说，梯度是相对于图像的。

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # 获得损失相对于输入图像的梯度
  gradient = tape.gradient(loss, input_image)
  # 获得创建扰动的梯度的符号
  signed_grad = tf.sign(gradient)
  return signed_grad

产生的扰动可以可视化。

In [ ]:
# 获得图像的输入标签
labrador_retriever_index = 208
label = tf.one_hot(labrador_retriever_index, image_probs.shape[-1])
label = tf.reshape(label, (1, image_probs.shape[-1]))

perturbations = create_adversarial_pattern(image, label)
plt.imshow(perturbations[0]*0.5+0.5); # To change [-1, 1] to [0,1]

可以尝试不同的系数，并观察结果图像。你将发现，随着系数epsilon的增加，欺骗网络变得容易。但是，这需要在欺骗网络和图像容易辨认间取得平衡。

In [ ]:
def display_images(image, description):
  _, label, confidence = get_imagenet_label(pretrained_model.predict(image))
  plt.figure()
  plt.imshow(image[0]*0.5+0.5)
  plt.title('{} \n {} : {:.2f}% Confidence'.format(description,
                                                   label, confidence*100))
  plt.show()

In [ ]:
epsilons = [0, 0.01, 0.1, 0.15]
descriptions = [('Epsilon = {:0.3f}'.format(eps) if eps else 'Input')
                for eps in epsilons]

for i, eps in enumerate(epsilons):
  adv_x = image + eps*perturbations
  adv_x = tf.clip_by_value(adv_x, -1, 1)
  display_images(adv_x, descriptions[i])

## 下一步

现在你已经找到对抗攻击，在不同的数据集和架构上试试。你可以创建和训练你自己的模型，然后使用相同的方法欺骗它。你也可以看看随着你调整系数，预测的置信度epsilon如何变化

尽管强大，本节展示的攻击只是对抗攻击研究的开端，有很多后面论文创建更强大的攻击。除了对抗攻击外，研究也导致防卫的产生，它的目的是创建强壮的机器学习模型。你可以参考 [survey paper](https://arxiv.org/abs/1810.00069)，全面了解对抗攻击和防卫。 

对于更多的对抗攻击和方位的实现，你可以看看对抗样本库[CleverHans](https://github.com/tensorflow/cleverhans).